In [1]:
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

BASE_DIR = Path().resolve().parent.parent.parent
gps_path = BASE_DIR / "data" / "gps_all"
save_path = BASE_DIR / "AI 모델" / "1.모델소스코드" / "3.여행루트추천" / "data"
print(gps_path)
print(save_path)


C:\Users\User\Desktop\xodls\through_pjt_ai\data\gps_all
C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\3.여행루트추천\data


In [2]:
from glob import glob
gps_list = glob(str(gps_path) + "/*.csv")
print(len(gps_list))
print(gps_list[0])

1774
C:\Users\User\Desktop\xodls\through_pjt_ai\data\gps_all\tn_gps_coord_h_h000001.csv


In [ ]:
import os
import sys
from functools import lru_cache
from dotenv import load_dotenv
import requests

load_dotenv("../.env")
KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

In [ ]:
# CT1 : 문화시설
# AT4 : 관광명소
# AD5 : 숙박
# FD6 : 음식점
# CE7 : 카페

In [ ]:
@lru_cache(maxsize=50000)
def reverse_geocode(lat, lon):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    params = {"y": lat, "x": lon}

    res = requests.get(url, headers=headers, params=params).json()

    if res.get("documents"):
        address = None
        if res["documents"][0]["road_address"]:
            address = res["documents"][0]["road_address"]
        elif res["documents"][0]["address"]:
            address = res["documents"][0]["address"]
            
        if address:
            add_dict = dict()
            add_dict["ADDRESS_FULL"] = address["address_name"]
            add_dict["ADDRESS_1DEPTH"] = address["region_1depth_name"]
            add_dict["ADDRESS_2DEPTH"] = address["region_2depth_name"]
            add_dict["ADDRESS_3DEPTH"] = address["region_3depth_name"]
            return add_dict
    else:
        return None

@lru_cache(maxsize=50000)
def get_nearby_poi(lat, lon, category):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}
    params = {
        "y": lat,
        "x": lon,
        "radius": 70,   # 50m 반경 내 장소 검색
        "category_group_code": category
    }
    res = requests.get(url, headers=headers, params=params).json()
    docs = res.get("documents", [])
    if docs:
        return docs[0]["place_name"]
    return None


In [ ]:
@lru_cache(maxsize=50000)
def gps_to_place(lat, lon): # 긴거, 짧은거
    # 1) 주소 변환
    address = reverse_geocode(lat, lon)

    # 2) 장소 검색 (관광명소 > 문화시설 > 음식점 > 카페 > 숙박 순서)
    for cat in ["AT4", "CT1", "FD6", "CE7", "AD5"]:
        poi = get_nearby_poi(lat, lon, cat)
        if poi and address:
            address["ADDRESS_NAME"] = poi
            return address
        
    else:
        if address:
            address["ADDRESS_NAME"] = None
            return address
        return None
    
@lru_cache(maxsize=50000)
def gps_to_place_cached(lat, lon):
    return gps_to_place(lat, lon)


In [ ]:
import pandas as pd
print(gps_list[60])
test = pd.read_csv(gps_list[60])
print(test.head(10))

In [ ]:
# 상하 범위 : 33.114115402 - 33.56448276039651
# 좌우 범위 : 126.14625579156079 - 126.969676444

In [ ]:
import numpy as np
def haversine(lon1, lat1, lon2, lat2):
    """
    위도/경도(도 단위) 2점 사이 거리(km)
    """
    R = 6371  # 지구 반지름 (km)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arcsin(np.sqrt(a))
    return R * c

def classify_row(row,
                 stay_radius_km=0.5,      # 체류 반경 (~200m)
                 stay_min_min=10,         # 체류 최소 시간
                 move_speed_kmh=15):       # 이동으로 볼 속도 기준
    if pd.isna(row['DIST_KM']) or pd.isna(row['DELTA_MIN']):
        return 'start'   # 여행 시작점
    # 체류
    if (row['DIST_KM'] <= stay_radius_km) and (row['DELTA_MIN'] >= stay_min_min):
        return 'stay'
    # 이동
    if row['SPEED_KMH'] >= move_speed_kmh:
        return 'move'
    # 그 외는 경유/애매지점
    return 'pass'




def csv_preprocessing(df):
    df = df.drop_duplicates("DT_MIN").drop_duplicates(["X_COORD","Y_COORD"])
    df = df[(df["X_COORD"].between(126.14625579156079, 126.969676444)) 
            &(df["Y_COORD"].between(33.114115402, 33.56448276039651))]
    # 공항 내 빼기
    df = df[~((df["X_COORD"].between(126.4891168819876, 126.501925142)) 
            &(df["Y_COORD"].between(33.50059843466816, 33.50769954833248)))]
    df['DT_MIN'] = pd.to_datetime(df['DT_MIN'])

    df = df.sort_values(['DT_MIN'])
    
    # # 10분 단위 슬롯
    # df['DT_10MIN'] = df['DT_MIN'].dt.floor('10min')

    # # 대표 지점: 여기선 평균 좌표 + 슬롯의 가장 이른 시간 사용
    # df_10 = (
    # df.groupby(['DT_10MIN'])
    #     .agg({
    #             'X_COORD': 'min',      # 경도 최빈
    #             'Y_COORD': 'min',      # 위도 최빈
    #             'DT_MIN': 'min'         # 슬롯 내 실제 첫 시각
    #     })
    #     .reset_index()
    #     .rename(columns={'DT_MIN': 'DT_REPR'})
    #     )

    # # 정렬 다시
    df_10 = df.sort_values(['DT_MIN'])# DT_REPR


    # 이전 포인트와 비교할 수 있도록 shift
    df_10[['X_PREV', 'Y_PREV', 'T_PREV']] = df_10[['X_COORD', 'Y_COORD', 'DT_MIN']].shift(1) # DT_REPR
            

    # 시간 차이 (분 단위)
    df_10['DELTA_MIN'] = (df_10['DT_MIN'] - df_10['T_PREV']).dt.total_seconds() / 60 # DT_REPR

    # 거리 (km)
    df_10['DIST_KM'] = haversine(
        df_10['X_PREV'], df_10['Y_PREV'],
        df_10['X_COORD'], df_10['Y_COORD']
        )

    # 속도 (km/h)
    df_10['SPEED_KMH'] = df_10['DIST_KM'] / (df_10['DELTA_MIN'] / 60)
    df_10['STATUS'] = df_10.apply(classify_row, axis=1)
    df_10 = df_10[df_10["STATUS"]=="stay"]

    
    
    return df_10.reset_index()[["MOBILE_NUM_ID", "X_COORD", "Y_COORD", "DT_MIN", "TRAVEL_ID"]]

In [ ]:
test1 = csv_preprocessing(test)

In [ ]:
test1.shape

In [ ]:
test1

In [ ]:
# 1. GPS → 장소 매핑
test1['address_info'] = test1.progress_apply(
    lambda row: gps_to_place_cached(row['Y_COORD'], row['X_COORD']),
    axis=1
)

# 2. None 제거
#test1 = test1[test1['address_info'].notna()].reset_index(drop=True)

# 3. dict 확장
test1_address = pd.json_normalize(test1['address_info'])

dup_cols = test1.columns.intersection(test1_address.columns)
test1_address = test1_address.drop(columns=dup_cols)

# 4. 원본과 합치기
test1 = pd.concat([test1, test1_address], axis=1)

# 5. 필요 없으면 address_info 컬럼 제거
test1.drop(columns=['address_info'], inplace=True)
cols = ["ADDRESS_FULL","ADDRESS_1DEPTH","ADDRESS_2DEPTH","ADDRESS_3DEPTH","ADDRESS_NAME"]

test1 = test1.dropna(subset=cols, how='all')
test1 = test1.drop_duplicates(["ADDRESS_NAME"])


In [ ]:
test1

# 실제 데이터 확장

In [ ]:
from tqdm import tqdm

In [ ]:
len(gps_list)

In [ ]:
for i, csv_file in enumerate(gps_list):
    fname = csv_file.split('_')[-1]
    if fname == "h000126.csv":
        print(i)
        break

In [ ]:
import gc
for i, csv_file in tqdm(enumerate(gps_list)):
    if i <= 71: continue
    try:
        fname = csv_file.split('_')[-1]
        test = pd.read_csv(csv_file)
        test1 = csv_preprocessing(test)
        # 1. GPS → 장소 매핑
        test1['address_info'] = test1.progress_apply(
            lambda row: gps_to_place_cached(row['Y_COORD'], row['X_COORD']),
            axis=1
        )

        # 2. None 제거
        #test1 = test1[test1['address_info'].notna()].reset_index(drop=True)

        # 3. dict 확장
        test1_address = pd.json_normalize(test1['address_info'])

        dup_cols = test1.columns.intersection(test1_address.columns)
        test1_address = test1_address.drop(columns=dup_cols)

        # 4. 원본과 합치기
        test1 = pd.concat([test1, test1_address], axis=1)

        # 5. 필요 없으면 address_info 컬럼 제거
        test1.drop(columns=['address_info'], inplace=True)
        cols = ["ADDRESS_FULL","ADDRESS_1DEPTH","ADDRESS_2DEPTH","ADDRESS_3DEPTH","ADDRESS_NAME"]

        test1 = test1.dropna(subset=cols, how='all')
        test1 = test1.drop_duplicates(["ADDRESS_NAME"]).reset_index(drop=True)
        test1['ADDRESS_NAME'] = test1['ADDRESS_NAME'].replace(["", " ", "None"], pd.NA)
        test1 = test1.dropna(subset=['ADDRESS_NAME'])
        test1['date'] = test1['DT_MIN'].dt.date
        # 여행별 day index
        test1['TRAVEL_DAY'] = test1['date'].transform(lambda x: x.ne(x.shift())).cumsum()
        test1.drop(columns=['date'], inplace=True)
        test1.to_csv(save_path / fname)
        # 큰 변수 삭제 및 가비지 컬렉션 호출
        # del large_variable
        gc.collect()
        
    except:
        print(i, fname)

In [ ]:
except_list = [
    69, 148, 170, 176, 193, 274, 276, 278, 284, 286, 
    287, 307, 367, 389, 417, 441, 459, 490, 546, 580, 
    623, 658, 681, 688, 747, 764, 784, 832, 841, 850, 
    852, 854, 870, 876, 879, 922, 929, 936, 955, 980,
    993, 1013, 1112, 1117, 1194, 1213, 1242, 1321, 1431, 1434,
    1437, 1448, 1499, 1518, 1541, 1545, 1547, 1549, 1554, 1555,
    1556, 1557, 1558, 1559, 1560, 1561, 1564, 1566, 1567, 1575,
    1576, 1577, 1579, 1580, 1581, 1584, 1585, 1587, 1588, 1589,
    1591, 1593, 1594, 1596, 1597, 1598, 1599, 1600, 1605, 1610, 
    1611, 1612, 1613, 1615, 1616, 1774,
    ]

### 위의 로직상의 'TRAVEL_DAY' 오류는 고쳤지만 시간이 오래걸리기에 아래 코드로 대신 수정

In [3]:
print(save_path)

C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\3.여행루트추천\data


In [4]:
csv_list = glob(str(save_path) + '/*.csv')

In [11]:
print(len(csv_list))
print(csv_list[0])

1675
C:\Users\User\Desktop\xodls\through_pjt_ai\AI 모델\1.모델소스코드\3.여행루트추천\data\h000001.csv


In [8]:
from tqdm import tqdm
def arrange_day(lst):
    tmp_lst = [0]*len(lst)
    tmp_day = 1
    for i, day in enumerate(lst):
        if i == 0:
            prev = day
            tmp_lst[i] = tmp_day
        else:
            if prev == day:
                tmp_lst[i] = tmp_day
            else:
                prev = day
                tmp_day += 1
                tmp_lst[i] = tmp_day
    else:
        return tmp_lst

In [10]:
a = [1,1,1,1,2,4,4,4,5,5]
print(arrange_day(a))

[1, 1, 1, 1, 2, 3, 3, 3, 4, 4]


In [13]:
import pandas as pd
for csv in tqdm(csv_list):
    df_c = pd.read_csv(csv)
    travel_day = df_c['TRAVEL_DAY'].to_list()
    df_c['TRAVEL_DAY'] = arrange_day(travel_day)
    df_c.to_csv(csv)
    

100%|██████████| 1675/1675 [00:35<00:00, 47.38it/s]
